# Optimal Power Flow Problem

Set up the objective function

<center> $\large \min\sum_{g} P(g)C(g)$

where $g$ is the index of the power generation source.</center>

Kirchoff's current law provides the constraints:



<center> $P_{g(n)} - \sum_{n=l(s)} P_l + \sum_{n=l(r)} P_l = P_{d(n)}\space\space\space \forall n$

$P_l = B_l(\theta_{l(n=s)}-\theta_{l(n=r)})\space\space\space \forall l$

$0 \le P_g \le \max(P_g)$

$-\max(P_l) \le P_l \le \max(P_l)$

$-\pi \le \theta_n \le \pi \space\space\space \forall n$</center>

In [1]:
!apt-get install -y coinor-cbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-2 [465 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-2 [275 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-3 [420 kB]
Get:5 ht

In [2]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [4]:
# Read in data

bus = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Optimization with Python/Exercises/powerSystem.xlsx', sheet_name='bus')
line = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Optimization with Python/Exercises/powerSystem.xlsx', sheet_name='line')
generation = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Optimization with Python/Exercises/powerSystem.xlsx', sheet_name='generation')
load = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Optimization with Python/Exercises/powerSystem.xlsx', sheet_name='load')

In [5]:
bus.head(10)

,bus
0,0
1,1
2,2


In [6]:
line.head(10)

,from_bus,to_bus,plmax,Bl
0,0,1,15,100
1,0,2,15,100
2,1,2,15,100


In [7]:
generation.head(10)

,bus,pgmax,cost
0,0,20,0.2
1,1,30,0.5


In [8]:
load.head(10)

,bus,load
0,2,25


In [9]:
n_bus,n_gen,n_line,n_load = len(bus),len(generation),len(line),len(load)

Set up the model and variables

In [10]:
model = pyo.ConcreteModel()
model.Pg = pyo.Var(range(n_gen))
model.Pl = pyo.Var(range(n_line)) # Corrected: Pl should be indexed by the number of lines
model.theta = pyo.Var(range(n_bus), bounds=(-np.pi,np.pi))

Pg = model.Pg
Pl = model.Pl
theta = model.theta

Set up the objective function

In [11]:
model.obj = pyo.Objective(expr=sum(Pg[g]*generation.cost[g] for g in generation.index), sense=minimize)

Power Balance

In [12]:

model.balance = pyo.ConstraintList()

for n in bus.index:
    sum_Pg = sum([Pg[g] for g in generation.index[generation.bus==n]])
    sum_Pls = sum([Pl[l] for l in line.index[line.from_bus==n]])
    sum_Plr = sum([Pl[l] for l in line.index[line.to_bus==n]])
    sum_Pd = sum([load.load[d] for d in load.index[load.bus==n]])
    model.balance.add(expr= sum_Pg - sum_Pls + sum_Plr == sum_Pd)


Power Flow

In [13]:
model.flux = pyo.ConstraintList()
for l in line.index:
    Bl = line.Bl[l]
    n_send = line.from_bus[l]
    n_rec = line.to_bus[l]
    delta_theta = theta[n_send]-theta[n_rec]
    model.flux.add(expr= Pl[l] == Bl*delta_theta)

Generator Limits

In [14]:
model.gen_limit = pyo.ConstraintList()
for g in generation.index:
    model.gen_limit.add(expr= Pg[g] <= generation.pgmax[g])

Power Flow Limits

In [15]:
#power flow limits
model.limflux = pyo.ConstraintList()
for l in line.index:
    model.limflux.add(expr= Pl[l] >= -line.plmax[l])
    model.limflux.add(expr= Pl[l] <= line.plmax[l])

Reference

In [16]:
#reference
model.ref = pyo.Constraint(expr= theta[0] == 0)

Invoke the Model Solver

In [17]:
#solve
opt = SolverFactory('cbc')
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 6.5, 'Upper bound': 6.5, 'Number of objectives': 1, 'Number of constraints': 15, 'Number of variables': 8, 'Number of nonzeros': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.043813467025756836}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [18]:
model.pprint()

3 Var Declarations
    Pg : Size=2, Index={0, 1}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  20.0 :  None : False : False :  Reals
          1 :  None :   5.0 :  None : False : False :  Reals
    Pl : Size=3, Index={0, 1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :   5.0 :  None : False : False :  Reals
          1 :  None :  15.0 :  None : False : False :  Reals
          2 :  None :  10.0 :  None : False : False :  Reals
    theta : Size=3, Index={0, 1, 2}
        Key : Lower              : Value : Upper             : Fixed : Stale : Domain
          0 : -3.141592653589793 :   0.0 : 3.141592653589793 : False : False :  Reals
          1 : -3.141592653589793 : -0.05 : 3.141592653589793 : False : False :  Reals
          2 : -3.141592653589793 : -0.15 : 3.141592653589793 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expressi